In [ ]:
#IMPORT LIBRARIES AND INTRODUCE USER-DEFINED FUNCTIONS

import numpy as np
import pandas as pd
import random
import ast
import csv
from sklearn import svm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import *
from sklearn.naive_bayes import *
from sklearn.metrics import confusion_matrix, classification_report
from ast import literal_eval
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit, GridSearchCV
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        pass
        #print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax
  

print('Libraries imported.')

Libraries imported.


In [ ]:
#MOUNT DRIVE

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#IMPORT DATASET

filepath1 = "/content/drive/My Drive/train_dataset.jsonl"
db=pd.read_json(filepath1, lines=True)
print('File imported')
#db.info

File imported


In [ ]:
#IMPORT TESTSET
filepath2 = "/content/drive/My Drive/test_dataset_blind.jsonl"
bs=pd.read_json(filepath2, lines=True)
print('File imported')
bs.info

In [ ]:
#GENERATE LIST OF BLIND DATA

blind_instructions = list ()
for i in range (0,3000):
    blind_instruction = bs.iloc[i,0]
    blind_instructions.insert(i, blind_instruction)

print('Blind_Instructions list created') 
print(blind_instructions[1]) 
print(len(blind_instructions))

Blind_Instructions list created
['mov edx 0', 'mov esi 0', 'mov edi 1', 'xor eax eax', 'call 0x8000e99', 'nop dword [rax]', 'mov edi 0', 'call 0x8000eaa', 'nop word [rax + rax]', 'test dl dl', 'jne 0x8001340', 'mov rdx rdi', 'mov r8 rbp', 'mov ecx 0', 'mov rsi r12', 'mov edi 0', 'xor eax eax', 'call loc.08000ed2', 'add rsp 0x40', 'pop rbx', 'pop rbp', 'pop r12', 'pop r13', 'pop r14', 'ret', 'mov rdx rdi', 'xor eax eax', 'mov r9 rbp', 'mov rsi r12', 'mov edi 0', 'call fcn.0800100a', 'cmp dword [rbx + 0x18] 0xffffffffffffffff', 'je 0x800101d', 'mov rdi qword [rbx + 0x10]', 'test rdi rdi', 'jne 0x8001258', 'mov eax dword [rbx + 0x1c]', 'test eax eax', 'je loc.08000ed2', 'cmp eax 2', 'jbe 0x80011d0', 'mov r9d 0', 'mov r8d 0', 'mov ecx 0x1e8', 'jmp 0x8000e83', 'mov r8d eax', 'mov ecx 0', 'jmp 0x8000ff5', 'mov rcx qword [rbx + 0x20]', 'mov edx 0', 'mov r8 rbp', 'mov rsi r12', 'mov edi 0', 'neg rcx', 'cmp eax 2', 'mov eax 0', 'cmove rdx rax', 'xor eax eax', 'call 0x80011fa', 'jmp loc.08000ed2

In [ ]:
#GENERATE LISTS OF SAMPLES AND LABELS

instructions = list()
optimization = list()
for i in range (0,3000):
    instruction= db.iloc[i,0]
    opt= db.iloc[i,1]
    instructions.insert(i, instruction)
    optimization.insert(i, opt)

print(instructions[1])
print(optimization[1])
print('Instructions and optimization lists created')
print(len(instructions))

['xor edx edx', 'cmp rdi rsi', 'mov eax 0xffffffff', 'seta dl', 'cmovae eax edx', 'ret']
H
Instructions and optimization lists created
3000


In [ ]:
#BALANCE THE DATASET

instructions = list()
optimization = list()
for i in range (0,30000):
    if db.opt[i]=='H':
    instructions.insert(i, db.iloc[i,0])
    optimization.insert(i, db.iloc[i,1])
    if(len(instructions)>1500):
        break
for i in range (0,30000):
    if db.opt[i]=='L':
        instructions.insert(i+1500, db.iloc[i,0])
        optimization.insert(i+1500, db.iloc[i,1])
        if(len(instructions)>=3000):
            break

print(len(instructions))
print(len(optimization))

3000
3000


In [ ]:
#GENERATE LIST OF SPLIT INSTRUCTIONS

instructions_split=[]
for i in range (0, len(instructions)):
    instruction_split=list()
    j=0
    for string in instructions[i]:
    #print(string)
    #print(len(string))
        for k in range (0, len(string.split())):
            tmp=string
            tmp_split=tmp.split()
            instruction_split.insert(j,tmp_split[k])
            j+=1
    instructions_split.append(instruction_split)
    
print(instructions_split[0])
print(instructions[0])

['xor', 'edx', 'edx', 'cmp', 'rdi', 'rsi', 'mov', 'eax', '0xffffffff', 'seta', 'dl', 'cmovae', 'eax', 'edx', 'ret']
['xor edx edx', 'cmp rdi rsi', 'mov eax 0xffffffff', 'seta dl', 'cmovae eax edx', 'ret']


In [ ]:
#GENERATE LIST OF MNEMONICS

mnemonics=[]
for i in range (0, len(instructions)):
    mnemonic=list()
    j=0
    for string in instructions[i]:
        tmp=string
        tmp_split=tmp.split()
        mnemonic.insert(j,tmp_split[0])
        j+=1
    mnemonics.append(mnemonic)  
    
print(mnemonics[0])
print(mnemonics[1])
print(mnemonics[2])

['xor', 'cmp', 'mov', 'seta', 'cmovae', 'ret']
['push', 'mov', 'push', 'mov', 'sub', 'test', 'mov', 'jne', 'test', 'jne', 'test', 'je', 'mov', 'call', 'cmp', 'sbb', 'not', 'and', 'add', 'pop', 'pop', 'ret', 'test', 'mov', 'jne', 'lea', 'mov', 'mov', 'call', 'test', 'mov', 'mov', 'mov', 'jne', 'mov', 'jmp', 'xor', 'jmp']
['movapd', 'mov', 'mov', 'mov', 'mov', 'call', 'movsd', 'movsxd', 'movsd', 'mov', 'mov', 'divsd', 'movsd', 'movsd', 'divsd', 'movsd', 'movsd', 'divsd', 'movsd', 'movsd', 'divsd', 'movsd', 'movsd', 'divsd', 'divsd', 'movsd', 'movsd', 'call', 'test', 'mov', 'je', 'test', 'jne', 'mov', 'cmp', 'je', 'jle', 'cmp', 'je', 'cmp', 'jne', 'mov', 'call', 'mov', 'xor', 'add', 'pop', 'pop', 'pop', 'pop', 'ret', 'movsd', 'movsd', 'movapd', 'movsd', 'mulsd', 'addsd', 'movsd', 'subsd', 'movsd', 'movsd', 'mulsd', 'movsd', 'xorpd', 'movsd', 'movsd', 'movapd', 'movsd', 'movsd', 'jmp', 'cmp', 'jne', 'mov', 'jmp', 'movsxd', 'xor', 'mov', 'shl', 'call', 'mov', 'cmp', 'jne', 'mov', 'jmp', 'mo

In [ ]:
#GENERATE CUT LIST OF MNEMONICS

mnemonics=[]
for i in range (0, len(instructions)):
    mnemonic=list()
    j=0
    k=10
    if len(instructions[i])>20:
        for string1 in instructions[i]:
            tmp1=string1
            tmp_split1=tmp1.split()
            mnemonic.insert(j,tmp_split1[0])
            j+=1
            if j>9:
                break
        for string2 in reversed(instructions[i]):
            tmp2=string2
            tmp_split2=tmp2.split()
            mnemonic.insert(k,tmp_split2[0])
            k+=1
            if k>19:
                break
    elif len(instructions[i])<=20:
        for string in instructions[i]:  
            tmp=string
            tmp_split=tmp.split()
            mnemonic.insert(j,tmp_split[0])
            j+=1
    mnemonics.append(mnemonic)  
    
print(mnemonics[0])

['xor', 'cmp', 'mov', 'seta', 'cmovae', 'ret']


In [ ]:
#CONVERT LISTS INTO LISTS OF STRINGS AND ARRAYS

ins = list()
mnem = list()
blind_ins = list()
for i in range (0,len(instructions)):
    ins.insert(i, str(instructions[i]))
    mnem.insert(i, str(mnemonics[i]))
    blind_ins.insert(i, str(blind_instructions[i]))

opt_array = np.asarray(optimization)

#print('Instructions lists converted into strings')
#print('Optimization list converted into array')
#print(ins[1])
print(blind_ins[0])
print(len(blind_ins[0]))
#print(len(blind_ins))

['push r15', 'push r14', 'push r13', 'push r12', 'push rbp', 'push rbx', 'sub rsp 8', 'mov r13 rdi', 'mov qword [rdi + 0x20] 0', 'mov byte [rdi + 0x30] 0', 'mov byte [rdi + 0x32] 0', 'mov byte [rdi + 0x31] 0', 'mov rax qword [rdi]', 'mov rbp qword [rax + 8]', 'test rbp rbp', 'je 0x80011e0', 'mov r12d 0', 'jmp 0x80010bc', 'mov r12 rbx', 'cmp dword [rbp] 4', 'ja 0x800118e', 'mov eax dword [rbp]', 'jmp qword [rax*8]', 'call 0x8001193', 'mov ebx 1', 'add rbx r12', 'cmp rbx 0xffffffffffffffff', 'je 0x80011a6', 'cmp r12 rbx', 'jbe 0x80011d1', 'mov edx 5', 'mov esi 0', 'mov edi 0', 'call 0x80011ba', 'mov rdx rax', 'mov esi 0', 'mov edi 1', 'mov eax 0', 'call 0x80011d1', 'mov rbp qword [arg_8h]', 'test rbp rbp', 'jne 0x80010b9', 'jmp 0x80011e5', 'mov ebx 0', 'mov qword [r13 + 0x18] rbx', 'add rsp 8', 'pop rbx', 'pop rbp', 'pop r12', 'pop r13', 'pop r14', 'pop r15', 'ret']
876


In [ ]:
#FEATURE EXTRACTION

#vectorizer = HashingVectorizer() # multivariate-Bernoulli
#vectorizer = CountVectorizer(ngram_range=(2,7), binary=True) # multinomial
vectorizer = TfidfVectorizer(ngram_range=(2,7), binary=True)
X_all = vectorizer.fit_transform(mnem)
y_all= opt_array
print(X_all.shape)
print(y_all.shape)
print(X_all[1])
print(y_all[1])
print(mnem[1])

In [ ]:
#TRAINING

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
          test_size=0.2, random_state=15)

print("Size of Train set: %d - Size of Test set: %d" %(X_train.shape[0],X_test.shape[0]))
#print('%d %s %s' %(id,str(y_train[id]),str(X_train[id])))
print(y_train)

Size of Train set: 2400 - Size of Test set: 600
['H' 'L' 'L' ... 'L' 'L' 'L']


In [ ]:
#SVM

model = svm.SVC(kernel='linear', C=1)
print('SVM Model created')
model.fit(X_train, y_train)

In [ ]:
#BERNOULLI 

model = BernoulliNB().fit(X_train, y_train)
print('Bernoulli Model created')

In [ ]:
#MULTINOMIAL

model = MultinomialNB().fit(X_train, y_train)
print('Multinomial Model created')

Multinomial Model created


In [ ]:
#ACCURACY, PRECISION, RECALL AND CONFUSION MATRIX

acc = model.score(X_test, y_test)
print("Accuracy: %.3f" %acc)
y_pred = model.predict(X_test)
print("\nConfusion Matrix: \n")
print("%s" %confusion_matrix(y_test, y_pred))
print("\n\t\t\tCalssification Report: \n")
print(classification_report(y_test, y_pred))

Accuracy: 0.950

Confusion Matrix: 

[[276  25]
 [  5 294]]

			Calssification Report: 

              precision    recall  f1-score   support

           H       0.98      0.92      0.95       301
           L       0.92      0.98      0.95       299

    accuracy                           0.95       600
   macro avg       0.95      0.95      0.95       600
weighted avg       0.95      0.95      0.95       600



In [ ]:
#PREDICTIONS

#create a list of predictions

predictions = list()
X_all_blind = vectorizer.transform(blind_ins)
h=0
l=0
for i in range (0, len(blind_instructions)):
    ynew = model.predict(X_all_blind[i])
    predictions.insert(i, ynew)
    if predictions[i] == 'H':
        h+=1
    elif predictions[i] == 'L':
        l+=1   

print(predictions)
print(len(predictions))
print(h)
print(l)

#Write those predictions on a csv file

with open('predictions.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(predictions)
csvFile.close()